# Model Scoring with Watson Machine Learning

This notebook should be run in a Watson Studio project, using Default Python 3.6 runtime environment. It requires service credentials to access the following Cloud services:
* Watson Machine Learning

The notebook will submit requests to the German Credit Risk model. The notebook assumes the model has been created/deployed to Watson Machine Learning.

#### Dependency Setup

In [1]:
!pip install --upgrade watson-machine-learning-client | tail -n 1

#### Configure Service Credentials

Update the two cells below with your Cloud API Key and your Watson Machine Learning service credentials.

In [2]:
WML_CREDENTIALS = {
  "apikey": "XycYiznO-hJ4nideySgm4bHOjuzaCAWuU8WNgOpWwms_",
  "iam_apikey_description": "Auto-generated for key  ",
  "iam_apikey_name": "Service credentials-1",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/6e8b529cf4cc4ab891306944b91199ec::serviceid:ServiceId-ec2d1bef-6024-434f-9801-dff7fdcb207a",
  "instance_id": "e3da0525-31af-4c9e-a2d2-3aad877dd536",
  "url": "https://us-south.ml.cloud.ibm.com"
}


#### Model Parameters

We use the same name for the Scikit model and the deployment to WML.

__Ensure that the two parameters match the model / deployment you have previously subscribed__

In [3]:
MODEL_NAME = "Spark German Risk Model"
DEPLOYMENT_NAME = "Spark German Risk Deployment"

#### Gather Model Information

In [4]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

wml_client = WatsonMachineLearningAPIClient(WML_CREDENTIALS)
wml_client.repository.list_models()

model_uid = None
wml_models = wml_client.repository.get_details()
for model_in in wml_models['models']['resources']:
    if MODEL_NAME == model_in['entity']['name']:
        model_uid = model_in['metadata']['guid']
        break

deployment_uid = None
deployment = None
scoring_url = None
wml_deployments = wml_client.deployments.get_details()
for deployment_in in wml_deployments['resources']:
    if DEPLOYMENT_NAME == deployment_in['entity']['name']:
        deployment_uid = deployment_in['metadata']['guid']
        scoring_url = deployment_in['entity']['scoring_url']
        deployment = deployment_in
        break

if model_uid is None:
    print("No model ...")
    
if deployment_uid is None:
    print("No Model deployment...")
    
print("Model id: {}".format(model_uid))
print("Deployment id: {}".format(deployment_uid))
print("Scoring URL: {}".format(scoring_url))

------------------------------------  -----------------------  ------------------------  ---------
GUID                                  NAME                     CREATED                   FRAMEWORK
0e47d7a8-6157-48b0-ac8a-71f05aa9cceb  Spark German Risk Model  2019-11-12T17:04:51.177Z  mllib-2.3
------------------------------------  -----------------------  ------------------------  ---------
Model id: 0e47d7a8-6157-48b0-ac8a-71f05aa9cceb
Deployment id: 95ab5f08-df78-4432-8f3a-d0835a875832
Scoring URL: https://us-south.ml.cloud.ibm.com/v3/wml_instances/e3da0525-31af-4c9e-a2d2-3aad877dd536/deployments/95ab5f08-df78-4432-8f3a-d0835a875832/online


### Send Requests to Model 

Now that we have some model monitors enabled, we will send some scoring requests through our model. This next section randomly selects 200 records from the data feed and sends those records to the model for predictions. This is enough to exceed the minimum threshold for records set in the previous section, which allows OpenScale to begin calculating fairness and drift.

In [5]:
import json
import random
import time

from IPython.utils import io


with io.capture_output() as captured:
    !wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/german_credit_feed.json -O german_credit_feed.json
!ls -lh german_credit_feed.json
   
with open('german_credit_feed.json', 'r') as scoring_file:
    scoring_data = json.load(scoring_file)

fields = scoring_data['fields']
values = []
for _ in range(200):
    values.append(random.choice(scoring_data['values']))
payload_scoring = {"fields": fields, "values": values}

scoring_response = wml_client.deployments.score(scoring_url, payload_scoring)

print('Number of scoring results:', len(scoring_response['values']))
time.sleep(10)

-rw-r----- 1 dsxuser dsxuser 3.0M Nov 13 14:13 german_credit_feed.json
Number of scoring results: 200


## Next steps


__Return to the workshop instruction book.__


## Credits

This notebook was adapted from the following sources:

* [Monitor Models Code Pattern](https://github.com/IBM/monitor-wml-model-with-watson-openscale)
* [OpenScale Labs](https://github.com/pmservice/OpenScale-Labs)
* [OpenScale Tutorials](https://github.com/pmservice/ai-openscale-tutorials)

#### Original Authors
* Eric Martens, is a technical specialist having expertise in analysis and description of business processes, and their translation into functional and non-functional IT requirements. He acts as the interpreter between the worlds of IT and business.
* Lukasz Cmielowski, PhD, is an Automation Architect and Data Scientist at IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.